Suzan Verberne 17.10.2023 1
Assignment 2: sequence labelling
Text mining course
This is a hand-in assignment for groups of two students. Send in via Brightspace before or on
Tuesday November 7:
• Submit your report as PDF and your python code as separate file. Don’t upload a zip file
containing the PDF (the Python code might be zipped if it consists of multiple files).
• Your report should not be longer than 3 pages.
• Do not copy text from sources (other groups, web pages, generative models such as
chatGPT). Turnitin is enabled and a large overlap will be reported to the Board of Examiners.
Goals of this assignment
• You can pre-process existing annotated text data into the data structure that you need for
classifier learning.
• You can perform hyperparameter optimization.
• You can perform a sequence labelling task with annotated data in Huggingface.
• You can evaluate sequence labelling with the suitable evaluation metrics.
Preliminaries
• You have followed the Huggingface tutorial on token classification
https://huggingface.co/learn/nlp-course/chapter7/2 and its preliminaries (exercises week 6
and 7).
• You have all the required Python packages installed and Python 3.10.
We are going to train an NER classifier for the task “Emerging and Rare entity recognition” from the
Workshop on Noisy User-generated Text (W-NUT). The description of the task can be found at
https://noisy-text.github.io/2017/emerging-rare-entities.html (I put the data itself on Brightspace)
Tasks
1. Download W-NUT_data.zip from the Brightspace assignment and unzip the directory. It
contains 3 IOB files: wnut17train.conll (train), emerging.dev.conll (dev),
emerging.test.annotated (test).
2. Convert the IOB data to the correct data structure for token classification in Huggingface
(words and labels like the conll2023 data in the tutorial) and align the labels with the tokens.
Note that since you are working with a custom dataset, the data conversion is a necessary
step for using the Huggingface training function.
Suzan Verberne 17.10.2023 2
3. Set up the evaluation correctly for the W-NUT test set, following the tutorial.
4. Fine-tune a model with the default hyperparameter settings on the train set and evaluate
the model on the test set. These are your baseline results.
5. Set up hyperparameter optimization with the AdamW optimizer. During optimization, use
the dev set as validation. After the model has been optimized, evaluate the result on the test
set.
6. Extend the evaluation function so that it shows the Precision, Recall and F-score for each of
the entity types (person, location, etc.) on the test set. Include the metrics for the B-label of
the entity type, the I-label, and the full entities. Look up the definitions of macro- and micro-
average scores and compute the macro- and micro average F1 scores over all entities.
Write a report of at most 3 pages in which you:
• describe the task and the data (give a few statistics. What are the entity types?) and briefly
describe two challenges of the task and the data.
• show your results:
o a results table with both the baseline results and the results after hyperparameter
optimization (do not report results on the dev set, only on the test set): a table with
Precision, Recall, F-score for both settings.
o a table with the results after hyperparameter optimization for the different entity
types (Precision, Recall, F-score for B, I, and the full entities), and the macro- and
micro F1 scores.
• write brief conclusions. Address the following questions:
o What is the effect of hyperparameter optimization on the quality of the model?
o What does the difference between scores for different entity types tell you?
o Where does the difference between macro- and micro-averaged F1 scores come
from?
Grading
Maximum 2 points for each of the following criteria:
• General: length correct (2-3 pages) and proper writing + formatting
• Description of the task and the data, with description of 2 challenges
• Baseline results with default hyperparameter settings and results with optimized
hyperparameter settings
• Results after hyperparameter optimization for the different entity types
• Sensible conclusions, briefly addressing the questions listed above.

In [49]:
import datasets
from transformers import Trainer
from transformers import TrainingArguments
from transformers import AutoModelForTokenClassification
from huggingface_hub import interpreter_login
from transformers import DataCollatorForTokenClassification
import evaluate
import numpy as np

In [50]:
names=[
    "O",
    "B-corporation",
    "I-corporation",
    "B-creative-work",
    "I-creative-work",
    "B-group",
    "I-group",
    "B-location",
    "I-location",
    "B-person",
    "I-person",
    "B-product",
    "I-product"
    ]

In [51]:
train, valid, test = 'wnut17train.conll', 'emerging.dev.conll', 'emerging.test.annotated'
path = 'W-NUT_data'

raw_datasets = {}
for file, name in zip([train, valid, test], ['train', 'validation', 'test']):
    id = 0
    raw_datasets[name] = {'id': [], 'tokens': [], 'ner_tags': []}
    with open(f'{path}/{file}', 'r') as f:
        tokens, ner_tags = [], []
        for line in f:
            try:
                token, ner_tag = line.split()
                tokens.append(token)
                ner_tags.append(names.index(ner_tag))
            except:
                raw_datasets[name]['id'].append([id for _ in range(len(tokens))])
                raw_datasets[name]['tokens'].append(tokens)
                raw_datasets[name]['ner_tags'].append(ner_tags)
                id += 1
                tokens, ner_tags = [], []
    raw_datasets[name] = datasets.Dataset.from_dict(raw_datasets[name])

raw_datasets = datasets.DatasetDict(raw_datasets)
display(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1287
    })
})

# Processing the data

In [52]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [53]:
tokenizer.is_fast

True

In [54]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)

In [55]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [56]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [57]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [58]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/3394 [00:00<?, ? examples/s]

Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

# Fine-tuning the model with keras

## Data collation

In [59]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Metrics

In [60]:
metric = evaluate.load("seqeval")

In [61]:

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [62]:
def compute_all_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return all_metrics

In [63]:
def write_tex_table(all_metrics, filename):
    with open(filename, 'w') as f:
        f.write("\\begin{table}[h]\n")
        f.write("\\centering\n")
        f.write("\\begin{tabular}{lrrrr}\n")
        f.write("\\toprule\n")
        f.write(" & \\textbf{Precision} & \\textbf{Recall} & \\textbf{F1} & \\textbf{Number} \\\\\n")
        f.write("\\midrule\n")
        for key, value in all_metrics.items():
            if "overall" not in key:
                f.write(f"{key.title()} & {value['precision']:.2f} & {value['recall']:.2f} & {value['f1']:.2f} & {value['number']} \\\\\n")
        f.write("\\midrule\n")
        f.write(" & \\textbf{Overall precision} & \\textbf{Overall recall} & \\textbf{Overall F1} & \\textbf{Overall accuracy} \\\\\n")
        f.write("\\midrule\n")
        f.write(f" & {all_metrics['overall_precision']:.2f} & {all_metrics['overall_recall']:.2f} & {all_metrics['overall_f1']:.2f} & {all_metrics['overall_accuracy']:.2f} \\\\\n")
        f.write("\\bottomrule\n")
        f.write("\\end{tabular}\n")
        f.write("\\end{table}\n")
    
def print_fancy_table(all_metrics):
    # display fancy table in ipython widget
    from IPython.display import display, HTML
    html = "<table>"
    html += "<tr><th></th><th>Precision</th><th>Recall</th><th>F1</th><th>Number</th></tr>"
    for key, value in all_metrics.items():
        if "overall" not in key:
            html += f"<tr><td>{key.title()}</td><td>{value['precision']:.2f}</td><td>{value['recall']:.2f}</td><td>{value['f1']:.2f}</td><td>{value['number']}</td></tr>"
    html += "<tr><th></th><th>Overall precision</th><th>Overall recall</th><th>Overall F1</th><th>Overall accuracy</th></tr>"
    html += f"<tr><td></td><td>{all_metrics['overall_precision']:.2f}</td><td>{all_metrics['overall_recall']:.2f}</td><td>{all_metrics['overall_f1']:.2f}</td><td>{all_metrics['overall_accuracy']:.2f}</td></tr>"
    html += "</table>"
    display(HTML(html))
    

## Defining the model

In [64]:
id2label = {i: label for i, label in enumerate(names)}
label2id = {v: k for k, v in id2label.items()}

In [65]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


##  Fine-tuning the model

In [66]:
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token can be pasted using 'Right-Click'.
Token is valid (permission: write).
Your token has been saved in your con

In [67]:
training_args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="no",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [68]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()
trainer.push_to_hub(commit_message="Training complete")

  0%|          | 0/1275 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/127 [00:00<?, ?it/s]

{'eval_loss': 0.3455404043197632, 'eval_precision': 0.6007751937984496, 'eval_recall': 0.3708133971291866, 'eval_f1': 0.45857988165680474, 'eval_accuracy': 0.9154937115333155, 'eval_runtime': 3.3283, 'eval_samples_per_second': 303.154, 'eval_steps_per_second': 38.157, 'epoch': 1.0}
{'loss': 0.203, 'learning_rate': 1.215686274509804e-05, 'epoch': 1.18}


  0%|          | 0/127 [00:00<?, ?it/s]

{'eval_loss': 0.3773612380027771, 'eval_precision': 0.6176470588235294, 'eval_recall': 0.4019138755980861, 'eval_f1': 0.48695652173913045, 'eval_accuracy': 0.9206850414771207, 'eval_runtime': 3.4132, 'eval_samples_per_second': 295.62, 'eval_steps_per_second': 37.209, 'epoch': 2.0}
{'loss': 0.0832, 'learning_rate': 4.313725490196079e-06, 'epoch': 2.35}


  0%|          | 0/127 [00:00<?, ?it/s]

{'eval_loss': 0.3647298216819763, 'eval_precision': 0.5763688760806917, 'eval_recall': 0.4784688995215311, 'eval_f1': 0.5228758169934641, 'eval_accuracy': 0.926036928017126, 'eval_runtime': 3.5037, 'eval_samples_per_second': 287.977, 'eval_steps_per_second': 36.247, 'epoch': 3.0}
{'train_runtime': 182.147, 'train_samples_per_second': 55.9, 'train_steps_per_second': 7.0, 'train_loss': 0.1257201654770795, 'epoch': 3.0}


'https://huggingface.co/rubinho/bert-finetuned-ner/tree/main/'

## Evaluating the model (baseline)

In [69]:
predictions = trainer.predict(tokenized_datasets["test"])
eval_preds = predictions.predictions, predictions.label_ids
all_metrics = compute_all_metrics(eval_preds)
write_tex_table(all_metrics, 'bert-finetuned-ner_baseline.txt')
print_fancy_table(all_metrics)

  0%|          | 0/161 [00:00<?, ?it/s]

,Precision,Recall,F1,Number
Corporation,0.15,0.15,0.15,66
Creative-Work,0.51,0.22,0.31,142
Group,0.35,0.15,0.21,165
Location,0.51,0.45,0.48,150
Person,0.75,0.44,0.56,429
Product,0.12,0.09,0.10,127
,Overall precision,Overall recall,Overall F1,Overall accuracy
,0.50,0.31,0.38,0.93


## Hyperparameter optimization

In [77]:
training_args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="no",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

def model_init(): 
    return AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)    

def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32]),
        "adam_beta1": trial.suggest_float("beta_1", 0.9, 0.999),
        "adam_beta2": trial.suggest_float("beta_2", 0.9, 0.999),
        "weight_decay": trial.suggest_float("weight_decay", 0.0, 0.1),
        "adam_epsilon": trial.suggest_float("adam_epsilon", 1e-9, 1e-7, log=True),
    }

trainer = Trainer(
    model=None,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    model_init=model_init,
)

best_run = trainer.hyperparameter_search(
    direction="minimize",
    backend="optuna",
    hp_space=optuna_hp_space,
    n_trials=2,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[I 2023-11-03 16:52:23,114] A new study created in memory with name: no-name-9fe29ea9-b5b6-4df8-9b5b-aec8cc8c2900
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/321 [00:00<?, ?it/s]

  0%|          | 0/127 [00:00<?, ?it/s]

c:\Users\Ruben\miniconda3\envs\torch\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Ruben\miniconda3\envs\torch\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.718856930732727, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_accuracy': 0.8816697886004816, 'eval_runtime': 3.5621, 'eval_samples_per_second': 283.257, 'eval_steps_per_second': 35.653, 'epoch': 1.0}


...

c:\Users\Ruben\miniconda3\envs\torch\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.4367040693759918, 'eval_precision': 0.5683890577507599, 'eval_recall': 0.2236842105263158, 'eval_f1': 0.3210300429184549, 'eval_accuracy': 0.8995986085094996, 'eval_runtime': 3.4024, 'eval_samples_per_second': 296.557, 'eval_steps_per_second': 37.327, 'epoch': 2.0}
{'loss': 0.1736, 'learning_rate': 1.1016193269757248e-06, 'epoch': 2.35}


  0%|          | 0/127 [00:00<?, ?it/s]

{'eval_loss': 0.4069167375564575, 'eval_precision': 0.5753138075313807, 'eval_recall': 0.32894736842105265, 'eval_f1': 0.4185692541856925, 'eval_accuracy': 0.9076799571849077, 'eval_runtime': 3.3404, 'eval_samples_per_second': 302.063, 'eval_steps_per_second': 38.02, 'epoch': 3.0}
{'train_runtime': 180.0553, 'train_samples_per_second': 56.549, 'train_steps_per_second': 7.081, 'train_loss': 0.22230459475049785, 'epoch': 3.0}


[I 2023-11-03 17:00:00,142] Trial 1 finished with value: 2.230510387323034 and parameters: {'learning_rate': 5.107507788705632e-06, 'per_device_train_batch_size': 8, 'beta_1': 0.9509822023671163, 'beta_2': 0.947724818069466, 'weight_decay': 0.000454495444088876, 'adam_epsilon': 8.455546642454333e-08}. Best is trial 0 with value: 0.8816697886004816.


## Retrain model with optimized hyperparameters

In [82]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)
    
trainer.train()
trainer.push_to_hub(commit_message="Training complete")

  0%|          | 0/321 [00:00<?, ?it/s]

  0%|          | 0/127 [00:00<?, ?it/s]

{'eval_loss': 0.3711134195327759, 'eval_precision': 0.5635135135135135, 'eval_recall': 0.4988038277511962, 'eval_f1': 0.5291878172588833, 'eval_accuracy': 0.9263580412095264, 'eval_runtime': 3.4128, 'eval_samples_per_second': 295.653, 'eval_steps_per_second': 37.213, 'epoch': 1.0}


  0%|          | 0/127 [00:00<?, ?it/s]

{'eval_loss': 0.3694494962692261, 'eval_precision': 0.55, 'eval_recall': 0.4868421052631579, 'eval_f1': 0.516497461928934, 'eval_accuracy': 0.9256622959593257, 'eval_runtime': 3.4993, 'eval_samples_per_second': 288.347, 'eval_steps_per_second': 36.293, 'epoch': 2.0}


  0%|          | 0/127 [00:00<?, ?it/s]

{'eval_loss': 0.3760727345943451, 'eval_precision': 0.559228650137741, 'eval_recall': 0.48564593301435405, 'eval_f1': 0.5198463508322664, 'eval_accuracy': 0.9257693336901258, 'eval_runtime': 3.5526, 'eval_samples_per_second': 284.015, 'eval_steps_per_second': 35.748, 'epoch': 3.0}
{'train_runtime': 115.3098, 'train_samples_per_second': 88.301, 'train_steps_per_second': 2.784, 'train_loss': 0.036124984051951, 'epoch': 3.0}


pytorch_model.bin:   0%|          | 0.00/431M [00:00<?, ?B/s]

'https://huggingface.co/rubinho/bert-finetuned-ner/tree/main/'

In [84]:
predictions = trainer.predict(tokenized_datasets["test"])
eval_preds = predictions.predictions, predictions.label_ids
all_metrics = compute_all_metrics(eval_preds)
write_tex_table(all_metrics, 'bert-finetuned-ner_optuna.txt')
print_fancy_table(all_metrics)

  0%|          | 0/161 [00:00<?, ?it/s]

,Precision,Recall,F1,Number
Corporation,0.16,0.20,0.18,66
Creative-Work,0.51,0.25,0.33,142
Group,0.37,0.16,0.23,165
Location,0.52,0.43,0.47,150
Person,0.77,0.46,0.57,429
Product,0.16,0.09,0.12,127
,Overall precision,Overall recall,Overall F1,Overall accuracy
,0.51,0.32,0.40,0.94
